In [1]:
#Aquiagregamos todas las librerias
import numpy as np
import math


In [2]:
def particle_swarm_optimization(obj_func, dim, bounds, num_particles, max_iter, inertia, cognitive, social):
    # Inicializar partículas
    positions = np.random.uniform(bounds[0], bounds[1], (num_particles, dim))
    velocities = np.random.uniform(-1, 1, (num_particles, dim))
    personal_best_positions = positions.copy()
    personal_best_scores = np.array([obj_func(pos) for pos in positions])
    global_best_position = personal_best_positions[np.argmin(personal_best_scores)]
    global_best_score = np.min(personal_best_scores)

    # PSO Loop
    for iter in range(max_iter):
        for i in range(num_particles):
            # Evaluar la posición actual
            fitness = obj_func(positions[i])
            
            # Actualizar mejor personal
            if fitness < personal_best_scores[i]:
                personal_best_scores[i] = fitness
                personal_best_positions[i] = positions[i]
            
            # Actualizar mejor global
            if fitness < global_best_score:
                global_best_score = fitness
                global_best_position = positions[i]

        # Actualizar velocidades y posiciones
        for i in range(num_particles):
            r1, r2 = np.random.rand(dim), np.random.rand(dim)  # Factores aleatorios
            cognitive_component = cognitive * r1 * (personal_best_positions[i] - positions[i])
            social_component = social * r2 * (global_best_position - positions[i])
            velocities[i] = inertia * velocities[i] + cognitive_component + social_component
            positions[i] = positions[i] + velocities[i]
            
            # Limitar posiciones a los límites dados
            positions[i] = np.clip(positions[i], bounds[0], bounds[1])

        # Imprimir progreso
        #print(f"Iteración {iter + 1}/{max_iter}, Mejor Fitness: {global_best_score}")

    return global_best_position, global_best_score


In [3]:
#En este modulo se insertan todasla funciones para testear que aparecen en tabla 1 de 
#The gradient evolution algorithm: A new metaheuristic
def Sphere(x):
    return sum([xi**2 for xi in x])
def WeightedSphere(x):
    return sum([(i+1)*xi**2 for i, xi in enumerate(x)])
def SumOfDifferentPower(x):
    return sum([abs(xi) ** (i + 1) for i, xi in enumerate(x)])
def StepFunction (x):
    return sum([math.floor(xi+0.5)**2 for xi in x])
def Schwefel1_2(x):
    sum_j = 0
    for i in range(1, len(x) + 1):
        sum_i = sum(x[:i])#Suma hasta el elemento i
        sum_j += sum_i**2
    return sum_j
def Schwefel2_2(x):
    return sum([abs(xi) for xi in x])+np.prod([abs(xi) for xi in x])
def rosenbrock(x):
    sum_i = 0
    #Se toma D-1 para que la función pueda quedar dentro de los limites del indice
    for i in range(len(x) - 1):
        sum_i += (x[i] - 1)**2 + 100 * (x[i+1] - x[i]**2)**2
    return sum_i
def easom(x):
    return -((-1)**len(x)) * np.prod(np.cos(x)**2)*np.exp(-np.sum((x - np.pi)**2))
def ackley(x):
    D = len(x)
    suma_cuadrados = sum([xi**2 for xi in x])
    suma_cosenos=sum([np.cos(2 * np.pi * xi) for xi in x])
    term1 = -20 * np.exp(-0.2 * np.sqrt(suma_cuadrados / D))
    term2 = -np.exp(suma_cosenos / D)
    return term1 + term2 + 20 + np.exp(1)
def griewank(x):
    suma_cuadrados = sum([xi**2 for xi in x])
    producto_cosenos = np.prod([np.cos(xi / np.sqrt(i+1)) for i, xi in enumerate(x)])
    return 1/4000*suma_cuadrados-producto_cosenos+1
def rastrigin(x):
    return sum([(xi**2 - 10 * np.cos(2 * np.pi * xi))+10 for xi in x])
def salomon(x):
    suma_cuadrados = sum([xi**2 for xi in x])
    return - np.cos(2 * np.pi * np.sqrt(suma_cuadrados)) + 0.1 * np.sqrt(suma_cuadrados)+1
def schwefel_2_3(x):
    return 418.9892*len(x)*sum([-xi*np.sin(np.sqrt(abs(xi))) for xi in x]) 
def Whitley(x):
    D = len(x)
    total_sum = 0
    for i in range(D):
        for j in range(D):
            y_ij = 100 * (x[j] - x[i]**2)**2 + (1 - x[j])**2
            total_sum += (y_ij**2 / 4000 - np.cos(y_ij) + 1)**2
    return total_sum

In [4]:
dim = 10  # Dimensión del problema
bounds = (-100, 100)  # Límites del espacio de búsqueda
num_particles = 20  # Número de partículas
max_iter = 3000  # Número de iteraciones
inertia = 0.7  # Factor de inercia
cognitive = 1.5  # Componente cognitivo
social = 1.5  # Componente social

In [5]:
best_position, best_score = particle_swarm_optimization(
    obj_func=Sphere,
    dim=dim,
    bounds=bounds,
    num_particles=num_particles,
    max_iter=max_iter,
    inertia=inertia,
    cognitive=cognitive,
    social=social
)

# Resultados
print("Mejor posición encontrada para Sphere :", best_position)
print("Fitness de la mejor posición para Sphere:", best_score)

Mejor posición encontrada para Sphere : [-2.37133494e-35 -4.20592872e-35  5.65517633e-35 -2.67610072e-35
 -8.95050464e-36 -2.36784088e-35 -3.95985437e-35 -7.16689431e-36
 -2.44849156e-35  6.64228062e-36]
Fitness de la mejor posición para Sphere: 4.0610897657679814e-69


Para una esfera 10 dimensional, una población de 20 y 300 iteraciones la mejor aproximaciòn fue de 1.03E-128 usando el algoritmo de DE,  con el algoritmo DE PSO En el paper tiene como mejor aproximación 1.08E-42,  

Para la función de Rastrigin
$$f(\mathbf{x}) = \sum_{i=1}^D \left( x_i^2 - 10 \cos(2\pi x_i) + 10 \right)$$
una población de 20 y 300 iteraciones maximas la mejor aproximaciòn fue de 0 con el algoritmo de GE, usando el algoritmo DE PSO En el paper tiene como mejor aproximación 2.96.  

In [6]:

best_position, best_score = particle_swarm_optimization(
    obj_func=rastrigin,
    dim=dim,
    bounds=bounds,
    num_particles=num_particles,
    max_iter=max_iter,
    inertia=inertia,
    cognitive=cognitive,
    social=social
)

# Resultados
print("Mejor posición encontrada para rastrigin:", best_position)
print("Fitness de la mejor posición para rastrigin:", best_score)

Mejor posición encontrada para rastrigin: [ 3.11848298e-10 -9.94958639e-01  9.94958638e-01 -5.23493012e-10
  1.36100367e-10 -1.23951076e-10 -1.05475657e-09  1.98545162e-10
 -1.46122736e-09  8.92117482e-10]
Fitness de la mejor posición para rastrigin: 1.9899181141865796


In [7]:
best_position, best_score = particle_swarm_optimization(
    obj_func=WeightedSphere,
    dim=dim,
    bounds=bounds,
    num_particles=num_particles,
    max_iter=max_iter,
    inertia=inertia,
    cognitive=cognitive,
    social=social
)

# Resultados
print("Mejor posición encontrada para WeightedSphere :", best_position)
print("Fitness de la mejor posición para WeightedSphere:", best_score)

Mejor posición encontrada para WeightedSphere : [ 1.63097275e-34  5.48596245e-35  6.03565475e-35  3.58564161e-36
  1.99512666e-35 -2.07850046e-35  1.30653701e-36  6.48580175e-35
  3.33314838e-35 -7.35479512e-35]
Fitness de la mejor posición para WeightedSphere: 1.2373279119415758e-67
